In [1]:
import pandas as pd
import numpy as np
import dspy
from dotenv import load_dotenv
import ast

In [2]:
load_dotenv()

False

In [3]:
dataset_name = "darrow-ai/LegalLensNER"

In [4]:
from datasets import load_dataset
raw_datasets = load_dataset(dataset_name)


/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
raw_datasets = raw_datasets.map(
    lambda x: {
        "tokens": ast.literal_eval(x["tokens"]),
        "ner_tags": ast.literal_eval(x["ner_tags"]),
    }
)

In [6]:
raw_datasets['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [7]:
raw_datasets["train"][2]

{'id': '009c87b5-8608-4518-9ec1-ac1a1709d831',
 'tokens': ['man',
  ',',
  'i',
  'cant',
  'believe',
  'what',
  'happened',
  'recently',
  '.',
  'some',
  'company',
  'got',
  'busted',
  'for',
  'breaking',
  'the',
  'can-spam',
  'act',
  '.',
  'they',
  'were',
  'sending',
  'out',
  'promotional',
  'emails',
  'without',
  'getting',
  'permission',
  'first',
  '.',
  'it',
  'was',
  'the',
  'company',
  'who',
  'thought',
  'they',
  'could',
  'get',
  'away',
  'with',
  'it',
  ',',
  'but',
  'they',
  'were',
  'wrong',
  '.',
  'they',
  'were',
  'doing',
  'this',
  'to',
  'regular',
  'folks',
  'like',
  'you',
  'and',
  'me',
  '.',
  'not',
  'cool',
  '.'],
 'ner_tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LAW',
  'I-LAW',
  'O',
  'O',
  'O',
  'B-VIOLATION',
  'I-VIOLATION',
  'I-VIOLATION',
  'I-VIOLATION',
  'I-VIOLATION',
  'I-VIOLATION',
  'I-VIOLATION',
  'I-VIOLATIO

In [8]:
print(raw_datasets['train'][0]['tokens'])
print(raw_datasets['train'][0]['ner_tags'])

['another', 'case', 'that', 'caught', 'my', 'attention', 'was', 'a', 'violation', 'of', 'employment', 'laws', 'by', 'a', 'popular', 'talent', 'agency', '.', 'they', 'were', 'found', 'guilty', 'of', 'requiring', 'their', 'non-union', 'employees', ',', 'including', 'the', 'plaintiff', ',', 'to', 'work', 'more', 'than', '31.33', 'hours', 'per', 'week', 'without', 'providing', 'overtime', 'compensation', '.', 'this', 'was', 'a', 'clear', 'violation', 'committed', 'on', 'their', 'non-union', 'employees', '.', 'its', 'disheartening', 'to', 'see', 'such', 'practices', 'in', 'the', 'entertainment', 'industry', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LAW', 'I-LAW', 'O', 'O', 'B-VIOLATED BY', 'I-VIOLATED BY', 'I-VIOLATED BY', 'O', 'O', 'O', 'O', 'O', 'O', 'B-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 'I-VIOLATION', 

In [9]:
def get_label_list(dataset):
    label_set = set()
    for data in dataset:
        labels = data["ner_tags"]
        label_set.update(labels)
    return list(label_set)

In [10]:
def tokenize_and_align_labels(examples, tokenizer, label_to_id):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True,
    )
    labels = []
    for i, example_labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        last_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != last_word_id:
                label_id = label_to_id.get(example_labels[word_id], -100)
                label_ids.append(label_id)
            else:
                label_ids.append(
                    -100
                )
            last_word_id = word_id
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
label_list = get_label_list(
    raw_datasets["train"]
)  # Assuming 'train' split exists and contains the labels

In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True, use_fast = True)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
# Create label_to_id mapping
label_to_id = {label: i for i, label in enumerate(label_list)}

In [14]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list)
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Tokenization and alignment of labels
tokenized_datasets = raw_datasets.map(
    lambda x: tokenize_and_align_labels(x, tokenizer, label_to_id), batched=True
)

Map: 100%|██████████| 617/617 [00:00<00:00, 4074.79 examples/s]


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments("distilbert-finetuned-ner",
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=10,
                         warmup_steps=500,
                         weight_decay=0.01)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
import evaluate


metric = evaluate.load("seqeval")

In [17]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results


In [18]:
from transformers import Trainer

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

                                                
 10%|█         | 89/890 [00:55<05:58,  2.24it/s]

{'eval_loss': 1.0579078197479248, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 75, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 75, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 75, 'eval_VIOLATION_precision': 0.0, 'eval_VIOLATION_recall': 0.0, 'eval_VIOLATION_f1': 0.0, 'eval_VIOLATION_number': 616, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.8041627469426152, 'eval_runtime': 11.9809, 'eval_samples_per_second': 51.499, 'eval_steps_per_second': 6.51, 'epoch': 1.0}


 20%|██        | 178/890 [01:37<05:02,  2.35it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
 20%|██        | 178/890 [01:49<05:02,  2.35it/s]

{'eval_loss': 0.35483449697494507, 'eval_LAW_precision': 0.024390243902439025, 'eval_LAW_recall': 0.013333333333333334, 'eval_LAW_f1': 0.017241379310344827, 'eval_LAW_number': 75, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 75, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 75, 'eval_VIOLATION_precision': 0.1680129240710824, 'eval_VIOLATION_recall': 0.33766233766233766, 'eval_VIOLATION_f1': 0.2243797195253506, 'eval_VIOLATION_number': 616, 'eval_overall_precision': 0.163408913213448, 'eval_overall_recall': 0.24851367419738407, 'eval_overall_f1': 0.1971698113207547, 'eval_overall_accuracy': 0.8875117591721543, 'eval_runtime': 11.639, 'eval_samples_per_second': 53.011, 'eval_steps_per_second': 6.702, 'epoch': 2.0}


                                                 
 30%|███       | 267/890 [02:41<04:23,  2.37it/s]

...

                                                 
100%|██████████| 890/890 [08:52<00:00,  2.37it/s]

{'eval_loss': 0.3735874891281128, 'eval_LAW_precision': 0.6504854368932039, 'eval_LAW_recall': 0.8933333333333333, 'eval_LAW_f1': 0.752808988764045, 'eval_LAW_number': 75, 'eval_VIOLATED BY_precision': 0.49193548387096775, 'eval_VIOLATED BY_recall': 0.8133333333333334, 'eval_VIOLATED BY_f1': 0.6130653266331659, 'eval_VIOLATED BY_number': 75, 'eval_VIOLATED ON_precision': 0.4, 'eval_VIOLATED ON_recall': 0.6133333333333333, 'eval_VIOLATED ON_f1': 0.48421052631578954, 'eval_VIOLATED ON_number': 75, 'eval_VIOLATION_precision': 0.4062111801242236, 'eval_VIOLATION_recall': 0.5308441558441559, 'eval_VIOLATION_f1': 0.4602392681210415, 'eval_VIOLATION_number': 616, 'eval_overall_precision': 0.43679163034001744, 'eval_overall_recall': 0.5957193816884662, 'eval_overall_f1': 0.5040241448692154, 'eval_overall_accuracy': 0.9258701787394168, 'eval_runtime': 11.6145, 'eval_samples_per_second': 53.123, 'eval_steps_per_second': 6.716, 'epoch': 10.0}


100%|██████████| 890/890 [08:53<00:00,  1.67it/s]

{'train_runtime': 533.1982, 'train_samples_per_second': 13.316, 'train_steps_per_second': 1.669, 'train_loss': 0.34481533029106226, 'epoch': 10.0}


TrainOutput(global_step=890, training_loss=0.34481533029106226, metrics={'train_runtime': 533.1982, 'train_samples_per_second': 13.316, 'train_steps_per_second': 1.669, 'total_flos': 927754419916800.0, 'train_loss': 0.34481533029106226, 'epoch': 10.0})

In [20]:
eval_result = trainer.evaluate()

100%|██████████| 78/78 [00:13<00:00,  5.94it/s]


In [21]:
{k: v for k, v in eval_result.items()}

{'eval_loss': 0.3735874891281128,
 'eval_LAW_precision': 0.6504854368932039,
 'eval_LAW_recall': 0.8933333333333333,
 'eval_LAW_f1': 0.752808988764045,
 'eval_LAW_number': 75,
 'eval_VIOLATED BY_precision': 0.49193548387096775,
 'eval_VIOLATED BY_recall': 0.8133333333333334,
 'eval_VIOLATED BY_f1': 0.6130653266331659,
 'eval_VIOLATED BY_number': 75,
 'eval_VIOLATED ON_precision': 0.4,
 'eval_VIOLATED ON_recall': 0.6133333333333333,
 'eval_VIOLATED ON_f1': 0.48421052631578954,
 'eval_VIOLATED ON_number': 75,
 'eval_VIOLATION_precision': 0.4062111801242236,
 'eval_VIOLATION_recall': 0.5308441558441559,
 'eval_VIOLATION_f1': 0.4602392681210415,
 'eval_VIOLATION_number': 616,
 'eval_overall_precision': 0.43679163034001744,
 'eval_overall_recall': 0.5957193816884662,
 'eval_overall_f1': 0.5040241448692154,
 'eval_overall_accuracy': 0.9258701787394168,
 'eval_runtime': 13.5628,
 'eval_samples_per_second': 45.492,
 'eval_steps_per_second': 5.751,
 'epoch': 10.0}